In [1]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import clickhouse_connect

load_dotenv('/home/jovyan/.env')

# Чтение параметров подключения
CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST')
CLICKHOUSE_PORT = int(os.getenv('CLICKHOUSE_PORT'))
CLICKHOUSE_DB = os.getenv('CLICKHOUSE_DB')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD')

try:
    client = clickhouse_connect.get_client(
        host=CLICKHOUSE_HOST,
        port=CLICKHOUSE_PORT,
        database=CLICKHOUSE_DB,
        username=CLICKHOUSE_USER,
        password=CLICKHOUSE_PASSWORD
    )
    print("Подключение к ClickHouse установлено!")
except Exception as e:
    print(f"Ошибка подключения: {str(e)}")

Подключение к ClickHouse: clickhouse:8123/analysis


In [ ]:
logins_count = client.query('SELECT COUNT(*) FROM logins').result_rows
user_info_count = client.query('SELECT COUNT(*) FROM user_info').result_rows

print(f"Текущее количество записей в logins: {logins_count[0][0]}")
print(f"Текущее количество записей в user_info: {user_info_count[0][0]}")

if logins_count[0][0] > 0 or user_info_count[0][0] > 0:
    print("Внимание: Некоторые данные уже загружены. Продолжить загрузку?")

In [ ]:
logins_csv_path = input("Укажите путь к logins.csv (например, /home/jovyan/data/logins.csv): ")
if not os.path.isfile(logins_csv_path):
        print(f"Файл не найден: {logins_csv_path}")
else:
    logins_df = pd.read_csv(logins_csv_path)
    logins_data = logins_df.to_dict('records')
    
    if not logins_data:
        print("Нет данных для загрузки.")
    
    client.execute(
        'INSERT INTO logins (Timestamp, GUID, OuterIP, InnerIP, OpenVPNServer, NgToken) VALUES',
        logins_data
    )
    print(f"Загружено {len(logins_data)} записей в таблицу logins.")

In [ ]:
user_info_path = input("Укажите путь к user_info.json (например, /home/jovyan/data/user_info.json): ")
if not os.path.isfile(user_info_path):
        print(f"Файл не найден: {user_info_path}")
else:
    with open(user_info_path, 'r') as f:
        user_info_data = json.load(f)

    if not user_info_data:
        print("Нет данных для загрузки.")

    client.execute(
        'INSERT INTO user_info (GUID, LastName, FirstName, MiddleName) VALUES',
        user_info_data
    )
    print(f"Загружено {len(user_info_data)} записей в таблицу user_info.")